In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow_addons.layers import TCN

# 读取实际波动率数据
df = pd.read_csv("./Data/S27FE-10K.csv")
volatility = df['volatility'].values.reshape(-1, 1)

# 数据归一化
scaler = MinMaxScaler()
scaled_vol = scaler.fit_transform(volatility)

# 构建滑动窗口特征
def create_dataset(data, time_steps=20):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)

time_steps = 20
X, y = create_dataset(scaled_vol, time_steps)
X = X[..., np.newaxis]  # 增加通道维度

# 划分训练集与测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# 构建 TCN 模型
model = models.Sequential([
    TCN(input_shape=(time_steps, 1), nb_filters=64, kernel_size=3, dropout_rate=0.1),
    layers.Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# 回调函数：EarlyStopping + 学习率调整
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-6)

# 训练模型
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)

# 预测
y_pred = model.predict(X_test)

# 反归一化
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

# 绘图
plt.figure(figsize=(12, 6))
plt.plot(y_test_inv, label='True Volatility')
plt.plot(y_pred_inv, label='Predicted Volatility', alpha=0.7)
plt.legend()
plt.title("Volatility Prediction using TCN")
plt.show()


ModuleNotFoundError: No module named 'tensorflow_addons'